In [1]:
import pandas as pd
import json

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
f = open('trial_task.json', encoding='utf-8')
json_object = json.load(f)

In [4]:
trial_task = pd.json_normalize(json_object)
trial_task.head(3)

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,..."


In [5]:
products = pd.json_normalize(json_object, record_path='products', meta=['order_id'])
products.head(3)

,product,price,quantity,order_id
0,ломтик июльского неба,450,1,11973
1,билет в Израиль,1000,3,11973
2,статуэтка Ленина,200,3,11973


In [6]:
trial_task.drop(['products'], axis='columns', inplace=True)
df = trial_task.merge(products, how='left', on='order_id')
df.head()

,order_id,warehouse_name,highway_cost,product,price,quantity
0,11973,Мордор,-70,ломтик июльского неба,450,1
1,11973,Мордор,-70,билет в Израиль,1000,3
2,11973,Мордор,-70,статуэтка Ленина,200,3
3,62239,хутор близ Диканьки,-15,билет в Израиль,1000,1
4,85794,отель Лето,-50,зеленая пластинка,10,2


In [7]:
# 1. Найти тариф стоимости доставки для каждого склада
df_1 = df.groupby(['order_id', 'warehouse_name', 'highway_cost'], as_index=False,)['quantity'].sum()
df_1['delivery_cost_to_the_warehouse'] = df_1['highway_cost'] / df_1['quantity']
df_1 = pd.DataFrame(df_1.groupby('warehouse_name')['delivery_cost_to_the_warehouse'].mean())
df_1

,delivery_cost_to_the_warehouse
warehouse_name,
Мордор,-10.0
гиперборея,-20.0
остров невезения,-5.0
отель Лето,-25.0
хутор близ Диканьки,-15.0


In [8]:
# 2. Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара 
# (представить как таблицу со столбцами 'product', 'quantity', 'income', 'expenses', 'profit')

# price, quantity
df_2 = df.groupby('product')['price', 'quantity'].agg({'price': 'mean', 'quantity': sum})

# income
df_2['income'] = df_2['price'] * df_2['quantity']

# expenses
expenses_product = df.merge(df_1.reset_index(), on='warehouse_name', how='left')
expenses_product['expenses'] = expenses_product['quantity'] * expenses_product['delivery_cost_to_the_warehouse']
expenses_product = pd.DataFrame(expenses_product.groupby('product')['expenses'].sum())
df_2 = df_2.join(expenses_product)

# profit
df_2['profit'] = df_2['income'] + df_2['expenses']

df_2

,price,quantity,income,expenses,profit
product,,,,,
автограф Стаса Барецкого,600.0,48,28800.0,-820.0,27980.0
билет в Израиль,1000.0,58,58000.0,-1050.0,56950.0
зеленая пластинка,10.0,61,610.0,-920.0,-310.0
ломтик июльского неба,450.0,42,18900.0,-670.0,18230.0
плюмбус,250.0,65,16250.0,-940.0,15310.0
подписка на suppi-блог,150.0,33,4950.0,-545.0,4405.0
статуэтка Ленина,200.0,68,13600.0,-935.0,12665.0


In [22]:
# 3. Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). 
# А также вывести среднюю прибыль заказов

df_3 = df.copy()
df_3['income'] = df_3['price'] * df_3['quantity']
df_3 = df_3.groupby(['order_id', 'highway_cost'], as_index=False)['income'].sum()
df_3['order_profit'] = df_3['income'] + df_3['highway_cost']
df_3 = df_3.drop(columns = ['highway_cost', 'income']).set_index('order_id')
df_3.head(10)

,order_profit
order_id,
124,705
1391,490
2091,1300
2108,200
2558,355
2751,-40
2930,720
4035,4125
5430,1120


In [23]:
# средняя прибыль заказов
df_3['order_profit'].mean()

1352.3

In [10]:
# 4. Составить табличку типа 'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse' 
# (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)

df_4 = df.merge(df_1.reset_index(), on='warehouse_name', how='left')
df_4['income'] = df_4['price'] * df_4['quantity']
df_4['expenses'] = df_4['quantity'] * df_4['delivery_cost_to_the_warehouse']
df_4['profit'] = df_4['income'] + df_4['expenses']
profit_of_warehouse = df_4.groupby('warehouse_name')['profit'].sum().reset_index().rename(columns = {'profit': 'product_of_warehouse'})
df_4 = df_4.groupby(['warehouse_name', 'product'], as_index=False)['quantity', 'profit'].sum()
df_4 = df_4.merge(profit_of_warehouse, on='warehouse_name', how='left')
df_4['percent_profit_product_of_warehouse'] = df_4['profit'] / df_4['product_of_warehouse'] * 100
# df_4['percent_profit_product_of_warehouse'] = df_4['percent_profit_product_of_warehouse'].apply(lambda x: round(x, 2))
df_4 = df_4.drop(columns=['product_of_warehouse'])
df_4

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse
0,Мордор,автограф Стаса Барецкого,4,2360.0,13.841642
1,Мордор,билет в Израиль,9,8910.0,52.258065
2,Мордор,зеленая пластинка,11,0.0,0.000000
3,Мордор,ломтик июльского неба,3,1320.0,7.741935
4,Мордор,плюмбус,6,1440.0,8.445748
5,Мордор,подписка на suppi-блог,8,1120.0,6.568915
6,Мордор,статуэтка Ленина,10,1900.0,11.143695
7,гиперборея,автограф Стаса Барецкого,12,6960.0,18.181818
8,гиперборея,билет в Израиль,21,20580.0,53.761755
9,гиперборея,зеленая пластинка,10,-100.0,-0.261233


In [25]:
# 5. Взять предыдущую табличку и отсортировать 'percent_profit_product_of_warehouse' по убыванию, 
# после посчитать накопленный процент. Накопленный процент - это новый столбец в этой табличке, который должен называться
# 'accumulated_percent_profit_product_of_warehouse'. 
# По своей сути это постоянно растущая сумма отсортированного по убыванию столбца 'percent_profit_product_of_warehouse'.


df_5 = pd.DataFrame(columns=df_4.columns.tolist() + ['accumulated_percent_profit_product_of_warehouse'])
for warehouse_name in df_4['warehouse_name'].unique():
    ds = df_4.loc[df_4['warehouse_name']==warehouse_name].sort_values(by='percent_profit_product_of_warehouse', ascending=False)
    ds['accumulated_percent_profit_product_of_warehouse'] = ds["percent_profit_product_of_warehouse"].cumsum()
    df_5 = pd.concat([df_5, ds], axis=0)
df_5

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse
1,Мордор,билет в Израиль,9,8910.0,52.258065,52.258065
0,Мордор,автограф Стаса Барецкого,4,2360.0,13.841642,66.099707
6,Мордор,статуэтка Ленина,10,1900.0,11.143695,77.243402
4,Мордор,плюмбус,6,1440.0,8.445748,85.68915
3,Мордор,ломтик июльского неба,3,1320.0,7.741935,93.431085
5,Мордор,подписка на suppi-блог,8,1120.0,6.568915,100.0
2,Мордор,зеленая пластинка,11,0.0,0.0,100.0
8,гиперборея,билет в Израиль,21,20580.0,53.761755,53.761755
7,гиперборея,автограф Стаса Барецкого,12,6960.0,18.181818,71.943574
10,гиперборея,ломтик июльского неба,13,5590.0,14.602926,86.546499


In [12]:
# 6. Присвоить A,B,C - категории на основании значения накопленного процента ('accumulated_percent_profit_product_of_warehouse'). 
# Если значение накопленного процента меньше или равно 70, то категория A.
# Если от 70 до 90 (включая 90), то категория Б. Остальное - категория C. Новый столбец обозначить в таблице как 'category'

df_6 = df_5.copy()
df_6['category'] = pd.cut(
    df_6['accumulated_percent_profit_product_of_warehouse'],
    bins=[-float('inf'), 70, 90, float('inf')] ,
    right = True,
    labels=['A', 'B', 'C']
)
df_6 = df_6.reset_index(drop=True)
df_6

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse,category
0,Мордор,билет в Израиль,9,8910.0,52.258065,52.258065,A
1,Мордор,автограф Стаса Барецкого,4,2360.0,13.841642,66.099707,A
2,Мордор,статуэтка Ленина,10,1900.0,11.143695,77.243402,B
3,Мордор,плюмбус,6,1440.0,8.445748,85.68915,B
4,Мордор,ломтик июльского неба,3,1320.0,7.741935,93.431085,C
5,Мордор,подписка на suppi-блог,8,1120.0,6.568915,100.0,C
6,Мордор,зеленая пластинка,11,0.0,0.0,100.0,C
7,гиперборея,билет в Израиль,21,20580.0,53.761755,53.761755,A
8,гиперборея,автограф Стаса Барецкого,12,6960.0,18.181818,71.943574,B
9,гиперборея,ломтик июльского неба,13,5590.0,14.602926,86.546499,B
